## Base de Dados
O objetivo final do projeto é usar uma base de dados com as relações de parentesco dentro do bairro de Surubeju do município de Monte Alegre - PA. No entanto, devido ao tempo de coleta desses dados, usarei um apenas uma amostra ou um conjunto de dados fake para a criação da visualização.

## Motivação
No referido bairro, no qual eu resido, existe uma peculiaridade de que os habitantes deste local tendem a constituir famílias com moradores do próprio bairro. Diante disso, criaram-se relações de parentesco peculiares, complexas e até mesmo desconhecidas pelos residentes. Com isso, decidi criar uma visualização dinâmica que demonstrasse todas as relações de parentesco presentes no bairro.

## Expectativa
O objetivo é conseguir plotar um gráfico de redes interativo, em que o usuário possa inserir o nome de duas pessoas e verificar a rede de parentesco entre elas.


## Esboços
Estão algumas amostras de dados com relações de parentesco e esboços de como será a visualização final.

In [21]:
import pandas as pd

import networkx as nx

import matplotlib.colors as mcolors

import dash
import dash_html_components  as html
import dash_cytoscape as cyto

In [22]:
#Coletando Data Frame do xls
full_data = pd.read_excel('familia.xlsx',index_col=0)
full_data.head()

,nome,apelido,pai_id,pai,mae_id,mae
id,,,,,,
1,Waldomiro Vasconcelos,NaN,NaN,NaN,NaN,NaN
2,Waldomira Vasconcelos,NaN,NaN,NaN,NaN,NaN
3,Waldomira Vasconcelos,Miroca,1.0,Waldomiro Vasconcelos,2.0,Waldomira Vasconcelos
4,Kelly Regina Vasconcelos,Branca,98.0,Pedro Bandeira,3.0,Waldomira Vasconcelos
5,Cláudio Vasconcelos,Bloco,98.0,Pedro Bandeira,3.0,Waldomira Vasconcelos


In [23]:
name_by_id = full_data[["nome"]].to_dict()["nome"]

In [24]:
# Contando o número de pessoas no banco de dados
num_people = len(full_data['nome'].unique())
# Criando uma lista de cores disponíveis
color_list = list(mcolors.cnames.keys())
# Criando uma lista de cores de acordo com o número de pessoas
chosen_colors = color_list[1:num_people+3]
# Dicionário das pessoas correspondentes a cada cor
colors_by_id = dict(zip(full_data['nome'].index,chosen_colors))
# Criando um dataframe com as pessoas e suas cores correspondentes.
# df_colors = pd.DataFrame({'classes':full_data['nome'].unique(), 'colors':chosen_colors})
# df_colors.head()


In [25]:
# Criando o Grafo
# Adicionando os nodes
family_tree = nx.DiGraph()
for idx in full_data.index:
    family_tree.add_node(idx,
                id= idx,
                label=name_by_id[idx], 
                color = colors_by_id[idx])

In [26]:
# Adicionando Edges
father_ids = full_data["pai_id"].values
mother_ids = full_data["mae_id"].values
name_ids = full_data["nome"].index.values
edges_list = []

for name_id, father_id, mother_id in zip(name_ids, father_ids, mother_ids):
    if pd.notna(father_id):
        edges_list.append((name_id, int(father_id)))
    if pd.notna(mother_id):
        edges_list.append((name_id, int(mother_id)))

        
family_tree.add_edges_from(edges_list)

In [27]:
# #Criando entidade do Grafo
# G = nx.DiGraph()
# # Adicionando Nós
# # Nessa etapa são adicionados os nós soltos com valores e cores
# for idx in full_data.index:
#    G.add_node(
#       df.loc[idx, 'filho'],
#          label=df.loc[idx, 'filho'], 
#          color=colors[df.loc[idx,'pai-mae']],
#          className=df.loc[idx, 'filho']
#       )
   
# # Nessa etapa, são estabelecidas as conexões entre os grafos
# G.add_edges_from(zip(df['filho'].values, df['pai-mae'].values),
#                  color='turquoise', 
#                  relation = 'filho de')


# for g in G.nodes():
#    G.edges(data=True)

In [28]:
cyto_family_nodes = nx.readwrite.json_graph.cytoscape_data(family_tree)['elements']['nodes']
cyto_family_edges = nx.readwrite.json_graph.cytoscape_data(family_tree)['elements']['edges']

In [35]:
cyto_family_nodes

[{'data': {'id': 1,
   'label': 'Waldomiro Vasconcelos',
   'color': 'antiquewhite',
   'value': 1,
   'name': '1'}},
 {'data': {'id': 2,
   'label': 'Waldomira Vasconcelos',
   'color': 'aqua',
   'value': 2,
   'name': '2'}},
 {'data': {'id': 3,
   'label': 'Waldomira Vasconcelos',
   'color': 'aquamarine',
   'value': 3,
   'name': '3'}},
 {'data': {'id': 4,
   'label': 'Kelly Regina Vasconcelos',
   'color': 'azure',
   'value': 4,
   'name': '4'}},
 {'data': {'id': 5,
   'label': 'Cláudio Vasconcelos',
   'color': 'beige',
   'value': 5,
   'name': '5'}},
 {'data': {'id': 6,
   'label': 'Nicanor Vasconcelos',
   'color': 'bisque',
   'value': 6,
   'name': '6'}},
 {'data': {'id': 7,
   'label': 'Simone Cristina Vasconcelos',
   'color': 'black',
   'value': 7,
   'name': '7'}},
 {'data': {'id': 8,
   'label': 'Valdmylla Vasconcelos',
   'color': 'blanchedalmond',
   'value': 8,
   'name': '8'}},
 {'data': {'id': 9,
   'label': 'Theo Bandeira',
   'color': 'blue',
   'value': 9,
  

In [39]:
cyto.load_extra_layouts()

In [51]:
# Melhores layouts
# breadthfirst
# dagre
# klay

app = dash.Dash("Family Network")
app.layout = html.Div([ 
    html.P('Dash Cytoscape:'),
    cyto.Cytoscape(
        id ='cytoscape',
        elements=cyto_family_nodes+cyto_family_edges,
        layout={'name':'klay',
                #  'roots':'#1, #2'
                },
        style={'width':'100vw', 'height':'90vh'},
        stylesheet=[
            {'selector': 'dagre', 
                'style': { 'label': 'data(label)'}, 
                "text-wrap": "wrap"}
            ]
    )
    
])



app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2021 20:42:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 20:42:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 20:42:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 20:42:07] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


<img src='Grafo.png' width="400" height="400">